# SQL Optimizer in SQL
This notebook accompanies the extra [homework assignment](README.md) for CS286A, Fall 2017. Please see that document for instructions on installation and setup.

First, load support for the %sql magic in Jupyter, and log into the database.
You will have to do this every time you start the kernel in this Jupyter notebook.

In [ ]:
%load_ext sql
%sql postgresql://vagrant:vagrant@:5432/optimizer

## Setup 1. Create metadata tables for our optimizer
This section sets up all the information needed for our optimizer as a set of metadata tables. You can see a [diagram of the full schema here.](img/metaschema.png)

### Create a separate schema to hold the optimizer's tables and views
SQL schemas are like filesystem directories, except they cannot be nested. Here we will separate out the optimizer tables and views by creasing a new schema, `sqlopt`, and setting the database `search_path` to default to `sqlopt`. All the tables and views we define below will be under that schema. By default, in postgresql the search path starts with `public`. So if you connect to this database from psql you will have to reference the optimizer tables with the prefix `sqlopt.`, as in `SELECT * FROM sqlopt.catalog_tables`. More info on schemas is in the [PostgreSQL documentation](https://www.postgresql.org/docs/current/static/ddl-schemas.html).

In [ ]:
%%sql
DROP SCHEMA IF EXISTS sqlopt CASCADE;
CREATE SCHEMA sqlopt;
SET search_path to sqlopt,public;

-- Postgres doesn't include a built-in array sorting function.
-- We define one here. You do not need to understand this in any detail.
CREATE OR REPLACE FUNCTION array_sort(anyarray)
RETURNS anyarray AS $$
  SELECT ARRAY(SELECT unnest($1) ORDER BY 1)
$$ LANGUAGE sql;

-- Test the sorting function
CREATE TABLE test_array(rownumber integer, words text[]);
INSERT INTO test_array VALUES (1, '{the, quick, brown}'), (2, '{four, score, and, seven}');
SELECT rownumber, array_sort(words) AS sorted FROM test_array;

### System: metadata about the hardware and software configuration
The only state we'll keep about our "hardware configuration" is the size of the buffer, the number of buffers available for our query operators (sorts, hash joins, etc.) and "CPU factor" in the Selinger cost model. For our software configuration, we'll keep the type of our join methods. 

System information tables have the prefix `system_`.

In [ ]:
%%sql
DROP TABLE IF EXISTS system_hardware CASCADE;
DROP TABLE IF EXISTS system_join_methods CASCADE;

--
-- Machine Config
--
-- bufpool is size of the buffer pool
-- qbufs is the number of pages of memory allocated to each operator
-- cpufactor is the constant from Selinger's paper
CREATE TABLE system_hardware(bufpool integer, qbufs integer, cpufactor float);
INSERT INTO system_hardware VALUES (10, 5, .001);

--
-- Query Operators
--
-- right_index_requirement is TRUE if the join algorithm requires the
-- join predicate to match an index.
CREATE TABLE system_join_methods(name text PRIMARY KEY, right_index_requirement bool);
INSERT INTO system_join_methods VALUES 
  ('BNLJ', false), 
  ('INLJ', true), 
  ('Hash', false), 
  ('Merge', false);

### Catalog metadata: tables that store information about tables.
We'll need to keep track of the user's schema and basic statistics about their data. For this implementation, we'll just use the simple Selinger statistics.

All of our catalog tables with start with the prefix `catalog_`.

In [ ]:
%%sql
DROP TABLE IF EXISTS catalog_tables CASCADE;
DROP TABLE IF EXISTS catalog_columns CASCADE;
DROP TABLE IF EXISTS catalog_access_methods CASCADE;

-- tables and columns
CREATE TABLE catalog_tables(table_name text PRIMARY KEY, ntups integer, npages integer);
CREATE TABLE catalog_columns(table_name text, column_id integer, name text, type text, num_vals integer,
                        lo integer, hi integer,
                        PRIMARY KEY (table_name, column_id),
                        FOREIGN KEY (table_name) REFERENCES catalog_tables);

-- access methods: current limitation: single-column search keys
CREATE TABLE catalog_access_methods(am_id integer, table_name text, 
                            column_id integer, type text,
                            PRIMARY KEY (am_id),
                            FOREIGN KEY (table_name, column_id) 
                            REFERENCES catalog_columns);

### Parser metadata: information about the query to be optimized.
Here we define tables that hold information about the query to be optimized. You should assume these tables are populated by a parser for SQL; for testing we'll populate them by hand.

We only handle simple single-query blocks of the form
```SQL
   SELECT *
     FROM <query_tables>
    WHERE <T1.col> <op> <constant> (selections)
      AND <T1.col> = <T2.col> (two-relation equijoins)
```
All the query-specific state has the `query_` prefix.

In [ ]:
%%sql

DROP TABLE IF EXISTS query_tables CASCADE;
DROP TABLE IF EXISTS query_predicates CASCADE;

-- tables referenced
CREATE TABLE query_tables(table_name text PRIMARY KEY,
                          FOREIGN KEY (table_name) REFERENCES catalog_tables);

-- Assuming the WHERE clause is in CNF, each of the conjuncts
-- is called a "predicate" (Selinger calls it a "Boolean Factor").
-- We currently do not handle OR or NOT predicates, only conjunctions of single-table
-- and two-table predicates.
-- Note the CHECK constraint that ensures each row holds either single-table or 
-- two-table information.
CREATE TABLE query_predicates(
    pred_id integer PRIMARY KEY, 
    table_left text NOT NULL, column_left integer NOT NULL,
    operator char(2), constant integer,        -- for selection predicates
    table_right text, column_right integer, -- for join predicates
    FOREIGN KEY (table_left, column_left) REFERENCES catalog_columns,
    FOREIGN KEY (table_right, column_right) REFERENCES catalog_columns,
    FOREIGN KEY (table_left) REFERENCES query_tables,
    FOREIGN KEY (table_right) REFERENCES query_tables,
    
    CHECK ((constant IS NULL AND operator IS NULL AND table_right IS NOT NULL AND column_right IS NOT NULL) 
           OR 
           (constant IS NOT NULL AND operator IS NOT NULL AND table_right IS NULL AND column_right IS NULL))
);

### The output of the optimization algorithm.
Here we register the schema of the output you need to generate from the Selinger algorithm. You should understand the structure of this table and its constraints well before you try to implement the optimizer algorithm.  This is a typical relational representation of a binary tree, in which each row represents a node in the tree with a unique `node_id`, and references other nodes. Here we're linked upward and downward: we have references to a `lhs` (left-hand-side) child, an `rhs` child, and a `parent` in each node. We also have explicit ordering among siblings captured in `sibling_id`, and a "level" of the node within the tree (leaves are at level 1, and parents are 1 level higher than their highest child.)

Metadata for the optimizer algorithms is prefixed with `opt_`.

In [ ]:
%%sql
--
-- Optimizer state
--
DROP TABLE IF EXISTS opt_chosen_plan;

-- The final chosen plan: your code needs to populate this table correctly.
CREATE TABLE opt_chosen_plan 
(
    node_id integer PRIMARY KEY,         -- id of this node
    level integer,       -- level in the tree (scan is 1)
    tables text[],       -- array of tables in this subtree
    am_id integer,       -- level=1: access method ID
    type text,           -- level=1: access method type
    top_join text,       -- level>1: join method
    lhs integer,         -- level>1: pid of left child
    rhs integer,         -- level>1: pid of right child,
    cost float,          -- cost
    npages integer,      -- number of pages of output
    ntups integer,       -- number of tuples of output
    order_table text, 
    order_column integer,  -- this column and the preceding define the ordering key of the output
    parent integer,      -- pid of the parent node
    sibling_id integer,  -- position in order of children of this parent (1 = left, 2 = right),
    FOREIGN KEY (am_id) REFERENCES catalog_access_methods,
    FOREIGN KEY (top_join) REFERENCES system_join_methods,
    FOREIGN KEY (lhs) REFERENCES opt_chosen_plan,
    FOREIGN KEY (rhs) REFERENCES opt_chosen_plan,
    FOREIGN KEY (order_table, order_column) REFERENCES catalog_columns,
    FOREIGN KEY (parent) REFERENCES opt_chosen_plan,
    
    CHECK (   (level = 1 AND am_id IS NOT NULL AND type IS NOT NULL)
           OR (level > 1 AND top_join IS NOT NULL AND lhs IS NOT NULL and rhs IS NOT NULL))
);

## Setup 2: Populating the Metadata
### Create Boating Club database
Here we define and populate an instance of the boating club database from the class examples. The actual data tables will be in the `public` schema. Since our `search_path` was set to have `sqlopt` first, we explicitly specify the `public` schema here.

This is the first time we embed SQL in Python code. Note the use of the one-line (`%sql`) syntax for `ipython-sql`, and the way we pass Python variable values into SQL queries via the `:` prefix.

In [ ]:
import os

%sql DROP TABLE IF EXISTS public.sailors CASCADE;
%sql DROP TABLE IF EXISTS public.boats CASCADE;
%sql DROP TABLE IF EXISTS public.reserves CASCADE;
%sql CREATE TABLE public.sailors(sid integer PRIMARY KEY, sname text, rating integer);
%sql CREATE TABLE public.boats(bid integer PRIMARY KEY, bname text, color text);
%sql CREATE TABLE public.reserves(rid integer PRIMARY KEY, sid integer, bid integer, rdate date, \
                      FOREIGN KEY (sid) REFERENCES sailors,\
                      FOREIGN KEY (bid) REFERENCES boats);
%sql CREATE INDEX ON public.reserves(bid);
%sql CREATE INDEX ON public.reserves(sid);

cwd = os.getcwd()
sailor_path = cwd + '/sailors.csv'
boat_path = cwd + '/boats.csv'
reserve_path = cwd + '/reserves.csv'
%sql COPY sailors FROM :sailor_path WITH (FORMAT csv);
%sql COPY boats FROM :boat_path WITH (FORMAT csv);
%sql COPY reserves FROM :reserve_path WITH (FORMAT csv);   
%sql ANALYZE;

### Populate the metadata: information about the class Boating Club database
Here we copy metadata from the PostgreSQL catalog into our own catalog tables.

In [ ]:
%%sql
INSERT INTO catalog_tables 
SELECT table_name, NULL::integer, NULL::integer
  FROM information_schema.tables
 WHERE table_schema = 'public';
    
INSERT INTO catalog_columns
SELECT table_name, ordinal_position, column_name, data_type, NULL::integer
  FROM information_schema.columns
 WHERE table_schema = 'public';

### Populate access methods.
Here we populate access methods for our tables:
- We assume there is a heap file for each table
- We also query PostgreSQL to see what indexes have been created, assuming each index is of type 'btree'. Note that PostgreSQL automatically builds an index on the primary key of a table.
This is the first time we use a SQL sequence number generator to autogenerate unique ids. You can read more about SQL sequences [in the PostgreSQL manual](https://www.postgresql.org/docs/current/static/sql-createsequence.html).

In [ ]:
%%sql
-- Sequence generator for access method ids
DROP SEQUENCE IF EXISTS catalog_am_seq CASCADE;
CREATE SEQUENCE catalog_am_seq;

INSERT INTO catalog_access_methods
SELECT nextval('catalog_am_seq'), table_name, NULL, 'heap'
  FROM catalog_tables;
    
INSERT INTO catalog_access_methods
SELECT nextval('catalog_am_seq'), t.relname, a.attnum, 'btree'
  FROM
    pg_class t,
    pg_class i,
    pg_index ix,
    pg_attribute a,
    pg_namespace ns
 WHERE
    t.oid = ix.indrelid
    and i.oid = ix.indexrelid
    and a.attrelid = t.oid
    and a.attnum = ANY(ix.indkey)
    and t.relkind = 'r'
    and t.relnamespace = ns.oid
    and ns.nspname = 'public';

### The next cell represents a parsed query, stored as a bunch of metadata.
Here we populate the query tables with parsed output for the following SQL query:
```sql
SELECT * 
  FROM sailors S, reserves R, boats B
 WHERE S.rating > 5 AND B.bid = 100 AND S.sid = R.sid AND R.bid = B.bid;
```

In [ ]:
%%sql

INSERT INTO query_tables VALUES ('sailors'), ('boats'), ('reserves');
INSERT INTO query_predicates VALUES 
    (1, 'sailors', 3, '>', 5, NULL, NULL), (2, 'boats', 1, '=', 100, NULL, NULL), 
    (4, 'sailors', 1, NULL, NULL, 'reserves', 2), (5, 'reserves', 3, NULL, NULL, 'boats', 1);

## Homework Part 1: Optimizer cost and selectivity formulae
Your first challenge is to define the optimizer cost formulae as PostgreSQL user-defined functions (UDFs). These functions should return type `numeric`, and can be implemented as simple SQL expressions that return a single row with a single column.  More information on PostgreSQL UDFs can be found [in the manual](https://www.postgresql.org/docs/current/static/xfunc-sql.html).

For simplicity we will not distinguish clustered and unclustered indexes; assume the cost of an index scan is the unclustered cost.

In [ ]:
%%sql
DROP FUNCTION IF EXISTS SeqScanCost(integer, integer) CASCADE;
DROP FUNCTION IF EXISTS IndexScanCost(integer, integer);
DROP FUNCTION IF EXISTS HashJoinCost(integer, integer, integer, integer, integer);
DROP FUNCTION IF EXISTS SortMergeJoinCost(integer, integer, integer, integer, integer);
DROP FUNCTION IF EXISTS BNLJCost(integer, integer, integer, integer, integer);
DROP FUNCTION IF EXISTS INLJCost(integer, integer, integer, integer, integer, integer);
DROP FUNCTION IF EXISTS SingleColumnSelectivity(char(2), integer, integer, integer);
DROP FUNCTION IF EXISTS TwoColumnEqSelectivity(integer, integer);

--
-- Scans
--
CREATE FUNCTION SeqScanCost(npages integer, ntups integer) RETURNS numeric AS $$
  -- WE ARE GIVING YOU THIS ONE FOR FREE :-)
  SELECT (npages + H.cpufactor*ntups)::numeric
    FROM system_hardware H
   LIMIT 1
$$ LANGUAGE SQL;

-- Unclustered cost only for now
CREATE FUNCTION IndexScanCost(npages integer, ntups integer) 
RETURNS numeric AS $$
    -- YOUR CODE HERE
$$ LANGUAGE SQL;


--
-- Joins
-- 
CREATE FUNCTION HashJoinCost(qbufs integer, lhspages integer, lhstups integer, 
                         rhspages integer, rhstups integer) RETURNS numeric AS $$
    -- YOUR CODE HERE
$$ LANGUAGE SQL;

CREATE FUNCTION SortMergeJoinCost(qbufs integer, lhspages integer, lhstups integer, 
                         rhspages integer, rhstups integer) RETURNS numeric AS $$
    -- YOUR CODE HERE
$$ LANGUAGE SQL;

DROP FUNCTION IF EXISTS BNLJCost(integer, integer, integer, integer, integer);
CREATE FUNCTION BNLJCost(qbufs integer, lhspages integer, lhstups integer, 
                         rhspages integer, rhstups integer) RETURNS numeric AS $$
    -- YOUR CODE HERE
$$ LANGUAGE SQL;

CREATE FUNCTION INLJCost(qbufs integer, lhspages integer, lhstups integer, 
                         rhspages integer, rhstups integer, rhs_numvals integer) 
                RETURNS numeric AS $$
    -- YOUR CODE HERE
$$ LANGUAGE SQL;


--
-- SingleColumnSelectivity
--
CREATE FUNCTION SingleColumnSelectivity(op char(2), constant integer, bin_lo integer, bin_hi integer) 
        RETURNS numeric AS $$
    -- YOUR CODE HERE
$$ LANGUAGE SQL;

--
-- TwoColumnEqSelectivity
--
CREATE FUNCTION TwoColumnEqSelectivity(lhs_height integer, rhs_height integer)
       RETURNS numeric AS $$
    -- YOUR CODE HERE
$$ LANGUAGE SQL;

## Homework Part 2: Populate the Optimizer Statistics
You are responsible to write queries to gather the statistics about tables and columns. 

This involves two basic tasks:
- Get the number of tuples and pages for each table. Assume that each page holds 4KB. Assume that integers are 4 bytes long, floats and dates are 8 bytes long, and strings are as many bytes as their length.
- gather distinct values, min and max (1-bucket histogram) for each integer column. For non-integer columns, min and max are set to NULL

Note that you'll have to run a query over each table to compute `ntups` and `npages`, and over each column of each table to compute `num_vals`, `lo` and `hi`. This is not possible in SQL, since SQL does not allow variables in the `FROM` clause. (That would be [second-order logic](https://en.wikipedia.org/wiki/Second-order_logic)). To achieve this you will have to embed SQL in Python, and pass Python variable values into your SQL. Be careful with your use of `$` vs. `:` in `ipython-sql`!

We've given you the SQL code to look up npages in the PostgreSQL catalog; you need to write the SQL code to compute `ntups`, and the column statistics. You should *not* take the statistics from the PostgreSQL catalog, you should compute them directly from the user tables. 



In [ ]:
tablenames = %sql select table_name from catalog_tables;
for t in tablenames:
    tname = t[0]
    %sql UPDATE catalog_tables \
            SET npages = (SELECT relpages \
                            FROM pg_class r, pg_namespace ns \
                           WHERE r.relnamespace = ns.oid \
                             AND r.relname = :tname \
                             AND ns.nspname = 'public') \
          WHERE table_name = :tname;

# YOUR CODE GOES HERE

## Homework Part 3: Dynamic Programming
Now that we have all our state set up, we can implement the core algorithm in SQL. 

First you will want to define your dynamic programming table and any ancillary tables or views you want to keep the state of the algorithm.

Hint: you may want to use the `CREATE SEQUENCE` feature to generate unique IDs for subplans found (and later reused) during dynamic programming.


In [ ]:
%%sql
-- Your DDL statements go here

Now you can write the DML queries for the dynamic program. Take it in two parts: base case and inductive case.

### Base case: find best single-table access plans
Here you need to write SQL to populate the dynamic programming table with level-1 information on scan subplans--both heap scans and index scans. We'll consider every heap scan, and any indexscan where the index key matches a column referenced in the query predicates.

We have given you a set of view skeletons to break the problem into pieces.

In [ ]:
%%sql

-- Extract all table_name references in predicates that could benefit from an index. 
-- There are two types of predicates:
-- 1) For predicates involving only one table, we want to know the 
--    table, column_id, selection operator and the constant.
-- 2) For predicates representing joins, we want to know tables 
--    and column_ids, as well as the join operator. Note that we're interested
--    in indexes both for the rhs and lhs of the predicate!
DROP VIEW IF EXISTS opt_index_refs CASCADE;
CREATE VIEW opt_index_refs(table_name, column_id, operator, constant) AS (
    -- YOUR CODE GOES HERE
);

-- Now combine the results of the previous view with rows that capture the case of 
-- heap scans that do not match a predicate.
-- Same schema as the previous view, but heap scans will have NULL info for 
-- columns, operators, and constants.
DROP VIEW IF EXISTS opt_table_col_refs CASCADE;
CREATE VIEW opt_table_col_refs(table_name, column_id, operator, constant) AS (
    -- YOUR CODE GOES HERE
);

-- Now find all the access methods corresponding to the references in the previous view.
DROP VIEW IF EXISTS opt_scan_access_methods CASCADE;
CREATE VIEW opt_scan_access_methods(table_name, column_id, am_id, type, npages, ntups, 
                                    order_column, operator, constant) AS (
    -- YOUR CODE GOES HERE
);

-- Compute costs for all full-scan access methods:
-- these are scans that return every tuple in the table. We should consider both 
-- heap and index access methods, as both types can return all tuples. The index
-- scans will produce interesting orders based on their search key.
DROP VIEW IF EXISTS opt_scan_AM_costs;
CREATE VIEW opt_scan_AM_costs(table_name, column_id, am_id, type, cost, npages, ntups, 
                              order_table, order_column, operator, constant) AS (
    -- YOUR CODE GOES HERE
);

-- Compute costs for index access methods that match a query predicate.
-- Cost formulas for indexes need to take predicate selectivity into account, 
-- since they only scan a fraction of the index and heap file. Again, these
-- return interesting orders.
DROP VIEW IF EXISTS opt_index_AM_costs CASCADE;
CREATE VIEW opt_index_AM_costs(table_name, column_id, am_id, type, cost, npages, ntups,
                               order_table, order_column) AS (
    -- YOUR CODE GOES HERE
);

-- Combine all of the level-1 access methods and their cost estimates
-- in a single view.
DROP VIEW IF EXISTS opt_all_scan_access_methods CASCADE;
CREATE VIEW opt_all_scan_access_methods(table_name, am_id, type, cost, npages, ntups,
                                        order_table, order_column) AS (
    -- YOUR CODE GOES HERE
);

-- Assign a unique ID to each AM.
-- We materialize this table_name so that each access method (subplan)
-- is assigned a persistent unique ID from a SQL sequence generator.
DROP TABLE IF EXISTS opt_numbered_scan_AMs CASCADE;
CREATE TABLE opt_numbered_scan_AMs(pid, table_name, am_id, type, cost, npages, ntups,
                                   order_table, order_column) AS (
    -- YOUR CODE GOES HERE
);

-- Find the minimum-cost plan for each interesting order (including NULL). You
-- will need to use an "argmin" style query pattern (see HW1).
-- Insert the results (with pids) into opt_bestplans, with level = 1

INSERT INTO opt_bestplans 
    -- YOUR CODE GOES HERE

### Inductive Case: form recursively larger join trees
In this cell we consider all ways of joining the n-1-way joins with the level-1 scans. Be sure to keep plans for all interesting orders! (Note that we are not handling GROUP BY or ORDER BY in this homework, so the only interesting orders related to joins.)

For each plan you consider, you need to compute cost, and use selectivity estimates to compute the output npages and ntups. You also need to track the output ordering based on the input. Currently we only track one column as an interesting order; for sort-merge join you can pick either of the join columns.

Make sure you consider "commuting" the join predicates: i.e. if you have a predicate of the form `R.x = S.y` you should also consider plans in which `R.x` is the right child of a join.

You can implement the induction by looping over values of n in Python, or by using a SQL [`WITH RECURSIVE`](https://www.postgresql.org/docs/9.6/static/queries-with.html) query.

In [ ]:
# YOUR CODE GOES HERE

### Now that we're done with iteration to find best join trees, let's choose the 1 best plan

In [ ]:
%%sql
-- Pick the root of the cheapest plan. If there are ties on cost, break arbitrarily
DROP VIEW IF EXISTS opt_final;
CREATE VIEW opt_final AS
-- YOUR CODE GOES HERE

Now given the root of the cheapest plan, construct the full plan tree with its
parent-child relationships, and sibling orders.  Start from the root and work downward.

This can be done with a single, relatively simple [`WITH RECURSIVE`](https://www.postgresql.org/docs/9.6/static/queries-with.html) query, or you can use a Python loop 
to issue multiple queries.

In [ ]:
%%sql
-- YOUR CODE GOES HERE

SELECT * FROM opt_chosen_plan;

At this point, the best plan is stored in `opt_best_plan`, and the database system can compile it into iterator objects to execute. You can use the [visualizer notebook](visualizer.ipynb) to see a graphical version of the plan.